Importation des librairies

In [1]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from unidecode import unidecode
import string

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 18.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import spacy

In [7]:
!pip install textblob_fr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.2/561.2 kB 6.4 MB/s eta 0:00:00


In [8]:
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import string
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
import plotly.graph_objects as go
import plotly.express as px

Recuperation des reponses du questionnaire dans le csv

In [9]:
csv_file = "dataQuestionnaire.csv"
data = pd.read_csv(csv_file)
texte_colonnes = ["Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)",
                 "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)",
                 "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)",
                 "Pouvez-vous expliquer vos réponses merci ..."]

In [10]:
pd.set_option('display.max_colwidth', None)
data.head()

,Séquentiel,SID,Heure de soumission,Heure de complétion,Heure de modification,Brouillon,Adresse IP,UID,Nom d'utilisateur,1 - Vous êtes ?,...,Les moyens financiers pérennes nécessaires à ces activités sont suffisants,Les équipe administratives et/ou techniques nécessaires à ces activités sont suffisantes,J'ai assez d'automie dans ces activités,"Quel serait pour vous, la répartition ""idéale"" entre Enseignement/Rech/Adm (donnez votre réponse sous la forme d'un ratio E/R/A - Ex. 45/45/10)",Pensez-vous avoir de bonnes chances d'avoir la RIPEC3 ?,"Pensez-vous avoir de bonnes chances de progresser dans votre carrière (changement de grade, de classe... ?",Finalement trouvez vous ce métier encore intéressant ?,Pensez-vous qu'il est attirant pour de jeunes chercheurs ?,Le conseilleriez-vous à un futur docteur en Informatique ?,Pouvez-vous expliquer vos réponses merci ...
0,1,15792852,3/31/2023 15:05,3/31/2023 15:05,3/31/2023 15:05,0,0.0.0.0,0,Anonymous,Une femme,...,0,0,4,45/10/45,Oui,Non,4.0,3.0,4.0,NaN
1,2,15896547,4/7/2023 13:32,4/7/2023 13:32,4/7/2023 13:32,0,0.0.0.0,0,Anonymous,Un homme,...,3,2,4,40/40/20,Non,Je ne suis pas ou plus concerné,4.0,4.0,4.0,NaN
2,3,15896551,4/7/2023 13:32,4/7/2023 13:32,4/7/2023 13:32,0,0.0.0.0,0,Anonymous,Un homme,...,3,3,3,4/4/2,Oui,Oui,5.0,2.0,4.0,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation)."
3,4,15896563,4/7/2023 13:33,4/7/2023 13:33,4/7/2023 13:33,0,0.0.0.0,0,Anonymous,Une femme,...,0,0,4,40/40/20,"Qui sait, sur un malentendu ...","Qui sait, sur un malentendu ...",3.0,1.0,1.0,NaN
4,5,15896653,4/7/2023 13:38,4/7/2023 13:38,4/13/2023 12:57,0,0.0.0.0,0,Anonymous,Une femme,...,2,2,4,33/33/33,"Qui sait, sur un malentendu ...",Non,4.0,2.0,4.0,"Je n'ai aucun espoir de passer un jour PU, ni même d'avoir la classe exceptionnelle, à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n"


Travail sur la premiere colonne contenant des commentaires

In [11]:
colonnes_a_garder = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)"]
df1 = data[colonnes_a_garder]
df1 = df1.dropna(subset="Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)")
df1.head()

,SID,Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)
1,15896547,"Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique."
10,15897993,Temps moyen hors période bouchon
14,15899015,"Temps moyen de déplacement selon les sites, peut aller de 5 minutes à 20 minutes"
15,15899810,"Je suis Responsable de M2, mon travail a une forte composante bureaucratique."
20,15901382,Pas de distance en temps pour transiter du lieu d'enseignement à celui de la recherche (site délocalisé) sauf pour les conseils de laboratoire qui sont sur le site central


In [12]:
def separer_texte(texte):
    morceaux = []
    morceau = ''
    for caractere in texte:
        if caractere in ['.', ',', ';', ':']:
            morceaux.append(morceau)
            morceau = ''
        else:
            morceau += caractere
    if morceau:
        morceaux.append(morceau)
    return morceaux

# Exemple d'utilisation :
texte = "Ceci est un texte. Il contient plusieurs phrases, séparées par des virgules; et des points-virgules."
resultat = separer_texte(texte)
print(resultat)

df1['Nouveaux morceaux'] = df1["Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)"].apply(separer_texte)
print(df1.head())

['Ceci est un texte', ' Il contient plusieurs phrases', ' séparées par des virgules', ' et des points-virgules']
         SID  \
1   15896547   
10  15897993   
14  15899015   
15  15899810   
20  15901382   

                                                                                                  Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)  \
1                                                                           Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.   
10                                                                                                                                             Temps moyen hors période bouchon   
14                                                                                             Temps moyen de déplacement selon les sites, peut aller de 5 minutes à 20 minutes   
15                                                                        

In [13]:
df_explose = df1.explode('Nouveaux morceaux')

colonnes_dupliquees = df_explose.apply(lambda x: x if x.name == 'Nouveaux morceaux' else x, axis=1)

colonnes_dupliquees.reset_index(drop=True, inplace=True)
colonnes_dupliquees.head()

,SID,Commentaires éventuels (sur le formulaire ou sur votre situation personnelle),Nouveaux morceaux
0,15896547,"Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.",Il n'y a plus d'UFR à l'Université de Bordeaux
1,15896547,"Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.",mais des UF
2,15896547,"Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.",Et en particulier une UF Informatique
3,15897993,Temps moyen hors période bouchon,Temps moyen hors période bouchon
4,15899015,"Temps moyen de déplacement selon les sites, peut aller de 5 minutes à 20 minutes",Temps moyen de déplacement selon les sites


In [14]:
nlp = spacy.load("fr_core_news_sm")

def extract_main_theme(text):
    doc = nlp(text)
    noun_chunks = [chunk.text for chunk in doc.noun_chunks if chunk.text.strip()]
    if noun_chunks:
        main_theme = max(set(noun_chunks), key=noun_chunks.count)
    else:
        main_theme = None

    return main_theme



colonnes_dupliquees["Main theme"] = colonnes_dupliquees["Nouveaux morceaux"].apply(extract_main_theme)
colonnes_dupliquees.head(20)

,SID,Commentaires éventuels (sur le formulaire ou sur votre situation personnelle),Nouveaux morceaux,Main theme
0,15896547,"Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.",Il n'y a plus d'UFR à l'Université de Bordeaux,Bordeaux
1,15896547,"Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.",mais des UF,mais des UF
2,15896547,"Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.",Et en particulier une UF Informatique,Informatique
3,15897993,Temps moyen hors période bouchon,Temps moyen hors période bouchon,Temps moyen
4,15899015,"Temps moyen de déplacement selon les sites, peut aller de 5 minutes à 20 minutes",Temps moyen de déplacement selon les sites,déplacement
5,15899015,"Temps moyen de déplacement selon les sites, peut aller de 5 minutes à 20 minutes",peut aller de 5 minutes à 20 minutes,20 minutes
6,15899810,"Je suis Responsable de M2, mon travail a une forte composante bureaucratique.",Je suis Responsable de M2,M2
7,15899810,"Je suis Responsable de M2, mon travail a une forte composante bureaucratique.",mon travail a une forte composante bureaucratique,mon travail
8,15901382,Pas de distance en temps pour transiter du lieu d'enseignement à celui de la recherche (site délocalisé) sauf pour les conseils de laboratoire qui sont sur le site central,Pas de distance en temps pour transiter du lieu d'enseignement à celui de la recherche (site délocalisé) sauf pour les conseils de laboratoire qui sont sur le site central,enseignement
9,15902826,UFR d'informatique 'propre' composée à 100% d'informaticiens,UFR d'informatique 'propre' composée à 100% d'informaticiens,UFR


In [15]:
from textblob import TextBlob

senti_list = []
for i in colonnes_dupliquees["Main theme"]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list.append('Positive')
        elif vs < 0:
            senti_list.append('Negative')
        else:
            senti_list.append('Neutral')
    else:
        senti_list.append('Neutral')  # Si l'élément n'est pas une chaîne de caractères, on considère le sentiment comme neutre

colonnes_dupliquees["Sentiment"] = senti_list
print(colonnes_dupliquees.head(20))

         SID  \
0   15896547   
1   15896547   
2   15896547   
3   15897993   
4   15899015   
5   15899015   
6   15899810   
7   15899810   
8   15901382   
9   15902826   
10  15903001   
11  15903001   
12  15903001   
13  15903001   
14  15903001   
15  15903160   
16  15903160   
17  15903369   
18  15904052   
19  15904052   

                                                                                                                             Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)  \
0                                                                                                      Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.   
1                                                                                                      Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.   
2                                            

In [16]:
from textblob import TextBlob

senti_list2 = []
for i in colonnes_dupliquees["Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)"]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list2.append('Positive')
        elif vs < 0:
            senti_list2.append('Negative')
        else:
            senti_list2.append('Neutral')
    else:
        senti_list2.append('Neutral')

colonnes_dupliquees["Sentiment2"] = senti_list2
print(colonnes_dupliquees.head())

        SID  \
0  15896547   
1  15896547   
2  15896547   
3  15897993   
4  15899015   

                         Commentaires éventuels (sur le formulaire ou sur votre situation personnelle)  \
0  Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.   
1  Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.   
2  Il n'y a plus d'UFR à l'Université de Bordeaux, mais des UF. Et en particulier une UF Informatique.   
3                                                                     Temps moyen hors période bouchon   
4                     Temps moyen de déplacement selon les sites, peut aller de 5 minutes à 20 minutes   

                                Nouveaux morceaux    Main theme Sentiment  \
0  Il n'y a plus d'UFR à l'Université de Bordeaux      Bordeaux   Neutral   
1                                     mais des UF   mais des UF   Neutral   
2           Et en particulier une UF Info

In [17]:
sentiment_counts = colonnes_dupliquees["Sentiment"].value_counts()
total_rows = len(colonnes_dupliquees)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     99.657534
Negative     0.342466
Name: Sentiment, dtype: float64


In [18]:
sentiment_counts = colonnes_dupliquees["Sentiment2"].value_counts()
total_rows = len(colonnes_dupliquees)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     82.876712
Positive    15.753425
Negative     1.369863
Name: Sentiment2, dtype: float64


In [19]:
colonnes_dupliquees.to_csv("commentaires1_TextBlob.csv", index=False)

Travail sur la deuxieme colonne contenant des commentaires

In [20]:
colonnes_a_garder = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)"]
df2 = data[colonnes_a_garder]
df2 = df2.dropna(subset="Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)")
df2.head()

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)
4,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche."
10,15897993,"A ma connaissance, mon enseignement n'est pas évalué."
13,15898762,"Attention, pour être reconnu, l'investissement pédagogique doit être évalué, et pour être évalué positivement, il vaut mieux faire ce qui est à la mode, ce qui va à l'encontre de la liberté pédagogique. Dans notre université, on a plein de message pour faire des approches ""récentes"" alors qu'on manque cruellement d'enseignants (la moitié des heures faites par des non permanents avec des thésards responsables de gros modules et aussi une forte rotation). \r\n"
14,15899015,"dernière question, ""S'il y a"" et non pas ""S'il n'y a"" c'est tout =p"
15,15899810,"Quand un jeune me demande si devenir prof de fac est intéressant, je lui conseille d'aller voir ailleurs. Les arguments développés sont \r\n- les conditions dramatiques d'enseignement (e.g. les TP par groupes de TD),\r\n- la disparition intégrale de toute liberté d'organisation personnelle (on ne nous a pas encore expliqué comment procéder pour pisser, mais ça va venir),\r\n- le mensonge éhonté du ""on vous recrute pour faire de la recherche"", alors que les soit disant chercheurs pratiquent en réalité le métier de VRP, pour lequel ils ne sont pas bons,\r\n- le fait que la recherche fondamentale est déclarée hors la loi,\r\n- le fait que les directions de thèses se font sous le contrôle de plein de comités Théodule... à quoi sert l'HDR ?\r\n- et je ne parle pas de la reconnaissance des actions conduites, quand les seuls qui obtiennent les promos sont les individualistes forcenés suffisamment lèche-cul pour se faire bien voir de qui il faut.\r\n\r\n"


In [21]:
df2['Nouveaux morceaux'] = df2["Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)"].apply(separer_texte)
print(df2.head())

         SID  \
4   15896653   
10  15897993   
13  15898762   
14  15899015   
15  15899810   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Commentaires éventuels (sur le formul

In [22]:
df_explose2 = df2.explode('Nouveaux morceaux')

# Dupliquer les autres colonnes
colonnes_dupliquees2 = df_explose2.apply(lambda x: x if x.name == 'Nouveaux morceaux' else x, axis=1)

# Réinitialiser l'index
colonnes_dupliquees2.reset_index(drop=True, inplace=True)
colonnes_dupliquees2.head()

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Nouveaux morceaux
0,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",L'enseignement est de plus en plus difficile
1,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent
2,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice)
3,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",je n'ai pas l'impression que cela sera aussi valorisé que la recherche
4,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",


In [23]:
colonnes_dupliquees2["Main theme"] = colonnes_dupliquees2["Nouveaux morceaux"].apply(extract_main_theme)
colonnes_dupliquees2.head(20)

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement),Nouveaux morceaux,Main theme
0,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",L'enseignement est de plus en plus difficile,L'enseignement
1,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent,des étudiants
2,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice),programmation
3,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",je n'ai pas l'impression que cela sera aussi valorisé que la recherche,cela
4,15896653,"L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.",,None
5,15897993,"A ma connaissance, mon enseignement n'est pas évalué.",A ma connaissance,A ma connaissance
6,15897993,"A ma connaissance, mon enseignement n'est pas évalué.",mon enseignement n'est pas évalué,mon enseignement
7,15898762,"Attention, pour être reconnu, l'investissement pédagogique doit être évalué, et pour être évalué positivement, il vaut mieux faire ce qui est à la mode, ce qui va à l'encontre de la liberté pédagogique. Dans notre université, on a plein de message pour faire des approches ""récentes"" alors qu'on manque cruellement d'enseignants (la moitié des heures faites par des non permanents avec des thésards responsables de gros modules et aussi une forte rotation). \r\n",Attention,Attention
8,15898762,"Attention, pour être reconnu, l'investissement pédagogique doit être évalué, et pour être évalué positivement, il vaut mieux faire ce qui est à la mode, ce qui va à l'encontre de la liberté pédagogique. Dans notre université, on a plein de message pour faire des approches ""récentes"" alors qu'on manque cruellement d'enseignants (la moitié des heures faites par des non permanents avec des thésards responsables de gros modules et aussi une forte rotation). \r\n",pour être reconnu,None
9,15898762,"Attention, pour être reconnu, l'investissement pédagogique doit être évalué, et pour être évalué positivement, il vaut mieux faire ce qui est à la mode, ce qui va à l'encontre de la liberté pédagogique. Dans notre université, on a plein de message pour faire des approches ""récentes"" alors qu'on manque c

In [24]:
senti_list3 = []
for i in colonnes_dupliquees2["Main theme"]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list3.append('Positive')
        elif vs < 0:
            senti_list3.append('Negative')
        else:
            senti_list3.append('Neutral')
    else:
        senti_list3.append('Neutral')  # Si l'élément n'est pas une chaîne de caractères, on considère le sentiment comme neutre

# Ajouter la liste des sentiments au DataFrame
colonnes_dupliquees2["Sentiment"] = senti_list3

# Afficher les premières lignes du DataFrame pour vérification
print(colonnes_dupliquees2.head(20))

         SID  \
0   15896653   
1   15896653   
2   15896653   
3   15896653   
4   15896653   
5   15897993   
6   15897993   
7   15898762   
8   15898762   
9   15898762   
10  15898762   
11  15898762   
12  15898762   
13  15898762   
14  15898762   
15  15898762   
16  15899015   
17  15899015   
18  15899810   
19  15899810   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [25]:
senti_list4 = []
for i in colonnes_dupliquees2["Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)"]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list4.append('Positive')
        elif vs < 0:
            senti_list4.append('Negative')
        else:
            senti_list4.append('Neutral')
    else:
        senti_list4.append('Neutral')  # Si l'élément n'est pas une chaîne de caractères, on considère le sentiment comme neutre

# Ajouter la liste des sentiments au DataFrame
colonnes_dupliquees2["Sentiment2"] = senti_list4

# Afficher les premières lignes du DataFrame pour vérification
print(colonnes_dupliquees2.head())

        SID  \
0  15896653   
1  15896653   
2  15896653   
3  15896653   
4  15896653   

                                                                                                                                                                                                                                                                                  Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de l'enseignement)  \
0  L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils travaillent. J'ai passé des centaines d'heures à codes des exercices de programmation en ligne sur Moodle (à raison d'une heure par exercice), je n'ai pas l'impression que cela sera aussi valorisé que la recherche.    
1  L'enseignement est de plus en plus difficile. J'ai des étudiants en 1ère année de BUT et j'ai trop souvent l'impression que je dois me battre pour qu'ils 

In [26]:
sentiment_counts = colonnes_dupliquees2["Sentiment"].value_counts()
total_rows = len(colonnes_dupliquees2)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     99.359658
Positive     0.640342
Name: Sentiment, dtype: float64


In [27]:
sentiment_counts = colonnes_dupliquees2["Sentiment2"].value_counts()
total_rows = len(colonnes_dupliquees2)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     71.718250
Positive    21.344717
Negative     6.937033
Name: Sentiment2, dtype: float64


In [28]:
colonnes_dupliquees2.to_csv("commentaires2_TextBlob.csv", index=False)

Travail sur la troisieme colonne contenant des commentaires

In [29]:
colonnes_a_garder = ["SID", "Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)"]
df3 = data[colonnes_a_garder]
df3 = df3.dropna(subset="Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)")
df3.head()

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)
4,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après."
5,15896801,Je suis dans une équipe Inria. Cela offre des conditions de recherche globalement bonnes.
10,15897993,Auparavant au CEA (R&D)\r\nMigration Université pour faire + de recherche et enseignement (interdit par hiérarchie du CEA)\r\nMis au placard des l'arrivée à l'Université (pas de responsabilité -> pas de promotion)\r\nRetour en arrière pas possible
12,15898367,"Les conditions d'exercices sont de plus en plus difficile. Outre la baisse des moyens récurrents, il est difficile d'obtenir des financements pour les doctorants ou post-doctorants étant emmenés à travailler sur des sujets ""amont"", ""pas à la mode"", ""exploratoires"", et , dans tous les cas, le salaire n'est jamais accompagné d'une part de fonctionnement (hors projets mais les sujets de recherche sont alors très contraints par les projets). Il devient donc difficile d'explorer des voies nouvelles, d'accompagner les doctorants dans la découverte du métier en leur permettant de faire leur thèse dans de bonnes conditions matérielles, humaines avec toutes les richesses qu'apportent les rencontres scientifiques en personne.\r\nSi on ajoute à cela le manque de reconnaissance du métier de chercheur (à tous les niveaux : institutionnel, industriel, sociétal, salarial, ...), on ne peut donc que constater la baisse inquiétante du vivier de recrutement."
13,15898762,Questionnement similaire à celui de Romain Couillet


In [30]:
df3['Nouveaux morceaux'] = df3["Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)"].apply(separer_texte)
print(df3.head())

         SID  \
4   15896653   
5   15896801   
10  15897993   
12  15898367   
13  15898762   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Commentaires éventuels (sur le formulaire

In [31]:
df_explose3 = df3.explode('Nouveaux morceaux')
colonnes_dupliquees3 = df_explose3.apply(lambda x: x if x.name == 'Nouveaux morceaux' else x, axis=1)
colonnes_dupliquees3.reset_index(drop=True, inplace=True)
colonnes_dupliquees3.head()

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche),Nouveaux morceaux
0,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée
1,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA
2,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile
3,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune
4,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",qui sont PU


In [32]:
colonnes_dupliquees3["Main theme"] = colonnes_dupliquees3["Nouveaux morceaux"].apply(extract_main_theme)
colonnes_dupliquees3.head(20)

,SID,Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche),Nouveaux morceaux,Main theme
0,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée,j'
1,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA,une équipe commune
2,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile,c'
3,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune,cette nouvelle équipe commune
4,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",qui sont PU,qui
5,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont PU, encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe. Quand on propose des sujets face à deux PU on passe forcément après.",encadrent des thèses entre pros et ne jouent plus leur rôle d'animateurs d'équipe,équipe
6,15896653,"Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été 

In [33]:
senti_list5 = []
for i in colonnes_dupliquees3["Main theme"]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list5.append('Positive')
        elif vs < 0:
            senti_list5.append('Negative')
        else:
            senti_list5.append('Neutral')
    else:
        senti_list5.append('Neutral')  # Si l'élément n'est pas une chaîne de caractères, on considère le sentiment comme neutre

# Ajouter la liste des sentiments au DataFrame
colonnes_dupliquees3["Sentiment"] = senti_list5

# Afficher les premières lignes du DataFrame pour vérification
print(colonnes_dupliquees3.head(20))

         SID  \
0   15896653   
1   15896653   
2   15896653   
3   15896653   
4   15896653   
5   15896653   
6   15896653   
7   15896653   
8   15896801   
9   15896801   
10  15897993   
11  15898367   
12  15898367   
13  15898367   
14  15898367   
15  15898367   
16  15898367   
17  15898367   
18  15898367   
19  15898367   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [34]:
senti_list6 = []
for i in colonnes_dupliquees3["Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)"]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list6.append('Positive')
        elif vs < 0:
            senti_list6.append('Negative')
        else:
            senti_list6.append('Neutral')
    else:
        senti_list6.append('Neutral')

colonnes_dupliquees3["Sentiment2"] = senti_list6
print(colonnes_dupliquees3.head())

        SID  \
0  15896653   
1  15896653   
2  15896653   
3  15896653   
4  15896653   

                                                                                                                                                                                                                                                                                                                                                                                                                                                  Commentaires éventuels (sur le formulaire ou sur votre appréciation personnelle de la recherche)  \
0  Je fais ma recherche quasiment toute seule depuis que l'équipe à laquelle j'étais rattachée a été démantelée, après qu'une partie des membres de cette équipe aient créé une équipe commune INRIA.\r\nJ'arrive tout de même à garder une activité de recherche mais c'est difficile. Je suis de plus assez choquée que certains membres de cette nouvelle équipe commune, qui sont P

In [35]:
sentiment_counts = colonnes_dupliquees3["Sentiment"].value_counts()
total_rows = len(colonnes_dupliquees3)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     99.360341
Positive     0.426439
Negative     0.213220
Name: Sentiment, dtype: float64


In [36]:
sentiment_counts = colonnes_dupliquees3["Sentiment2"].value_counts()
total_rows = len(colonnes_dupliquees3)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     74.840085
Positive    21.002132
Negative     4.157783
Name: Sentiment2, dtype: float64


In [37]:
colonnes_dupliquees3.to_csv("commentaires3_TextBlob.csv", index=False)

Travail sur la 4e colonne

In [38]:
colonnes_a_garder = ["SID", "Pouvez-vous expliquer vos réponses merci ..."]
df4 = data[colonnes_a_garder]
df4 = df4.dropna(subset="Pouvez-vous expliquer vos réponses merci ...")
df4.head()

,SID,Pouvez-vous expliquer vos réponses merci ...
2,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation)."
4,15896653,"Je n'ai aucun espoir de passer un jour PU, ni même d'avoir la classe exceptionnelle, à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n"
5,15896801,On garde l'espoir.
7,15897130,"Aujourd’hui, l’informatique offre une multitude de métiers. Le métier d’E/C est trop difficile, il faut être très motivé pour l’exercer sans frustration."
10,15897993,"Profession en perte de considération dans la société.\r\nConditions de travail en régression régulière.\r\nUn futur docteur est déjà engagé dans le processus, donc s'il a fait ce choix, pas de raison de le dissuader."


In [39]:
df4['Nouveaux morceaux'] = df4["Pouvez-vous expliquer vos réponses merci ..."].apply(separer_texte)
print(df4.head())

         SID  \
2   15896551   
4   15896653   
5   15896801   
7   15897130   
10  15897993   

                                                                                                                                                                                Pouvez-vous expliquer vos réponses merci ...  \
2                                                  Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).   
4                   Je n'ai aucun espoir de passer un jour PU, ni même d'avoir la classe exceptionnelle, à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n   
5                                                                                                                                                                                                         On garde l'espoir.   
7      

In [40]:
df_explose4 = df4.explode('Nouveaux morceaux')
colonnes_dupliquees4 = df_explose4.apply(lambda x: x if x.name == 'Nouveaux morceaux' else x, axis=1)
colonnes_dupliquees4.reset_index(drop=True, inplace=True)
colonnes_dupliquees4.head()

,SID,Pouvez-vous expliquer vos réponses merci ...,Nouveaux morceaux
0,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",Les conditions de travail dépendent beaucoup du lieu d'exercice
1,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",et peut fortement changer pour diverses raisons conjoncturelles (financements
2,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",maladie
3,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",réorganisation)
4,15896653,"Je n'ai aucun espoir de passer un jour PU, ni même d'avoir la classe exceptionnelle, à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n",Je n'ai aucun espoir de passer un jour PU


In [41]:
colonnes_dupliquees4["Main theme"] = colonnes_dupliquees4["Nouveaux morceaux"].apply(extract_main_theme)
colonnes_dupliquees4.head(20)

,SID,Pouvez-vous expliquer vos réponses merci ...,Nouveaux morceaux,Main theme
0,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",Les conditions de travail dépendent beaucoup du lieu d'exercice,Les conditions
1,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",et peut fortement changer pour diverses raisons conjoncturelles (financements,financements
2,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",maladie,maladie
3,15896551,"Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).",réorganisation),réorganisation
4,15896653,"Je n'ai aucun espoir de passer un jour PU, ni même d'avoir la classe exceptionnelle, à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n",Je n'ai aucun espoir de passer un jour PU,un jour
5,15896653,"Je n'ai aucun espoir de passer un jour PU, ni même d'avoir la classe exceptionnelle, à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n",ni même d'avoir la classe exceptionnelle,la classe exceptionnelle
6,15896653,"Je n'ai aucun espoir de passer un jour PU, ni même d'avoir la classe exceptionnelle, à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n",à cause du temps que je passe en enseignement et en administratif qui pénalise forcément mon temps de recherche\r\n,enseignement
7,15896801,On garde l'espoir.,On garde l'espoir,On
8,15897130,"Aujourd’hui, l’informatique offre une multitude de métiers. Le métier d’E/C est trop difficile, il faut être très motivé pour l’exercer sans frustration.",Aujourd’hui,Aujourd’hui
9,15897130,"Aujourd’hui, l’informatique offre une multitude de métiers. Le métier d’E/C est trop difficile, il faut être très motivé pour l’exercer sans frustration.",l’informatique offre une multitude de métiers,une multitude


In [42]:
senti_list7 = []
for i in colonnes_dupliquees4["Main theme"]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list7.append('Positive')
        elif vs < 0:
            senti_list7.append('Negative')
        else:
            senti_list7.append('Neutral')
    else:
        senti_list7.append('Neutral')

colonnes_dupliquees4["Sentiment"] = senti_list7
print(colonnes_dupliquees4.head(20))

         SID  \
0   15896551   
1   15896551   
2   15896551   
3   15896551   
4   15896653   
5   15896653   
6   15896653   
7   15896801   
8   15897130   
9   15897130   
10  15897130   
11  15897130   
12  15897130   
13  15897993   
14  15897993   
15  15897993   
16  15897993   
17  15897993   
18  15898332   
19  15898332   

                                                                                                                                                                                                                                                                          Pouvez-vous expliquer vos réponses merci ...  \
0                                                                                                                                            Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).   
1                                  

In [43]:
senti_list8 = []
for i in colonnes_dupliquees4["Pouvez-vous expliquer vos réponses merci ..."]:
    if isinstance(i, str):
        vs = TextBlob(i).sentiment[0]
        if vs > 0:
            senti_list8.append('Positive')
        elif vs < 0:
            senti_list8.append('Negative')
        else:
            senti_list8.append('Neutral')
    else:
        senti_list8.append('Neutral')

colonnes_dupliquees4["Sentiment2"] = senti_list8
print(colonnes_dupliquees4.head())

        SID  \
0  15896551   
1  15896551   
2  15896551   
3  15896551   
4  15896653   

                                                                                                                                                               Pouvez-vous expliquer vos réponses merci ...  \
0                                 Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).   
1                                 Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).   
2                                 Les conditions de travail dépendent beaucoup du lieu d'exercice, et peut fortement changer pour diverses raisons conjoncturelles (financements, maladie, réorganisation).   
3                                 Les conditions de travail dépendent beaucoup du

In [44]:
sentiment_counts = colonnes_dupliquees4["Sentiment"].value_counts()
total_rows = len(colonnes_dupliquees4)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     99.319971
Positive     0.501074
Negative     0.178955
Name: Sentiment, dtype: float64


In [45]:
sentiment_counts = colonnes_dupliquees4["Sentiment2"].value_counts()
total_rows = len(colonnes_dupliquees4)
percentages = (sentiment_counts / total_rows) * 100
print(percentages)

Neutral     60.952040
Positive    28.346457
Negative    10.701503
Name: Sentiment2, dtype: float64


In [46]:
colonnes_dupliquees4.to_csv("commentaires4_TextBlob.csv", index=False)